In [2]:
!pip install sagemaker==1.15.2

mkl-random 1.0.1 requires cython, which is not installed.
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
s3trainuri="s3://aegovansagemaker/ppirelation/data/training_gnorm_with_relation.xml"
trainfilename="training_gnorm_with_relation.xml"
bucket_name ="aegovansagemaker"



In [6]:
image="324346001917.dkr.ecr.us-east-2.amazonaws.com/ppi-relation-extraction:latest"

In [9]:
role = "arn:aws:iam::324346001917:role/service-role/AmazonSageMaker-ExecutionRole-20180111T103138"



In [4]:
from sagemaker import get_execution_role
import sagemaker


sagemaker_session = sagemaker.Session()



In [10]:
%%time
import boto3
from time import gmtime, strftime

job_name = 'ppi-lowlevel-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Training job", job_name)


output_location = 's3://{}/ppirelation/output'.format(bucket_name)
print('training artifacts will be uploaded to: {}'.format(output_location))

create_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": image,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": output_location
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.m4.xlarge",
        "VolumeSizeInGB": 70
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
                        'traindata':trainfilename       
                       ,"sagemaker_job_name"         :job_name 
                
                       },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60 * 24 * 5
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3trainuri,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }
       
    ]
}



sagemaker = boto3.client('sagemaker')

sagemaker.create_training_job(**create_training_params)

status = sagemaker.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print(status)

try:
    sagemaker.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=job_name)
finally:
    status = sagemaker.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    print("Training job ended with status: " + status)
    if status == 'Failed':
        message = sagemaker.describe_training_job(TrainingJobName=job_name)['FailureReason']
        print('Training failed with the following error: {}'.format(message))
        raise Exception('Training job failed')

Training job ppi-lowlevel-2018-11-26-09-44-31
training artifacts will be uploaded to: s3://aegovansagemaker/ppirelation/output
InProgress
Training job ended with status: Failed
Training failed with the following error: AlgorithmError: Exit Code: 2


Exception: Training job failed